In [18]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [19]:
## for Langsmith tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [20]:
## Data Ingestion - from the website we need to scrape the data 
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://python.langchain.com/docs/introduction/")
web_base_loader = loader.load()
web_base_loader

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\n\n\nSkip to main contentOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guides

In [21]:
#Dividing the document into chunks because the document is too long , llms have a limit on the number of tokens they can process
## Load data --> Docs --> Divide our text into chunks --> Vectors --> vector embeddings --> Store in vector database
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 800, chunk_overlap = 100) 
docs = text_splitter.split_documents(web_base_loader)

In [22]:
docs

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Skip to main contentOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval A

In [23]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [24]:
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(docs, embeddings)

In [25]:
vector_store

In [26]:
query = "integrations have been split into lightweight packages that"
result = vector_store.similarity_search(query, k=3)
result[0].page_content

"langchain-core: Base abstractions for chat models and other components.\nIntegration packages (e.g. langchain-openai, langchain-anthropic, etc.): Important integrations have been split into lightweight packages that are co-maintained by the LangChain team and the integration developers.\nlangchain: Chains, agents, and retrieval strategies that make up an application's cognitive architecture.\nlangchain-community: Third-party integrations that are community maintained.\nlanggraph: Orchestration framework for combining LangChain components into production-ready applications with persistence, streaming, and other key features. See LangGraph documentation."

In [27]:
#llms are used to generate the answer based on the context retrieved from the vector store
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000002E883A87B50> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002E883A87C70> root_client=<openai.OpenAI object at 0x000002E883A85030> root_async_client=<openai.AsyncOpenAI object at 0x000002E883A85330> model_name='gpt-4o-mini' model_kwargs={} openai_api_key=SecretStr('**********')


In [28]:
## Retrieval Chain 
## this is used to retrieve the relevant documents from the vector store based on the query
## context is the relevant documents retrieved from the vector store
## we pass the context to the llm to generate the answer
## this retrieval chain does this 
## previously we were using the vector store directly to get the relevant documents that match the query
## when we want to have a qna system we need to use a retrieval chain


In [ ]:
# This is the retrieval chain that we will use to get the relevant documents from the vector store
#create_stuff_documents_chain is used to create a chain that takes the context and the query and returns the answer
#here the context is - the relevant documents retrieved from the vector store

from langchain.chains.combine_documents import create_stuff_documents_chain

# This is the prompt template that we will use to generate the answer
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context: 
<context>
{context}
</context>



"""
)

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context: \n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002E883A87B50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002E883A87C70>, root_client=<openai.OpenAI object at 0x000002E883A85030>, root_async_client=<openai.AsyncOpenAI object at 0x000002E883A85330>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={},

Retriever will dynamically select the most relevant documents or chunks and pass those in for a given question , these selected chunks are known as context


In [31]:
## Retriver - its an interface that allows us to retrieve the relevant documents from the vector store based on the query
## we convert this vector store into a retriever
retriever = vector_store.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain) # retriever is used to retrieve the relevant documents from the vector store based on the query and document_chain is used to generate the answer based on the context retrieved from the vector store
#the context is retrieved from the retriever and passed to the document_chain to generate the answer
